Started to work on changes to make the modules usable by Githika

In [1]:
import sys
sys.path.append("/home/dquintero/venvs/serviceDSSAT/lib/python3.10/site-packages")
sys.path.append('/home/dquintero/dssat_service/dssat_service')

In [2]:
from dssatservice.database import connect
from dssatservice.dssat import run_spatial_dssat
from dssatservice.ui.base import SCHEMAS, admin_list, AdminBase, BASELINE_YEARS, Session

from highcharts_core.chart import Chart
# Funciton for validation charts
from dssatservice.ui.plot import validation_chart
# Functions for ColumnRange charts
from dssatservice.ui.plot import init_columnRange_chart, get_columnRange_series_data
# Functions for anomaly charts
from dssatservice.ui.plot import init_anomalies_chart, get_anomaly_series_data
#Function to clear yield charts (both columnrange and anomaly)
from dssatservice.ui.plot import clear_yield_chart
# Functions for stress charts
from dssatservice.ui.plot import init_stress_chart, get_stress_series_data, clear_stress_chart

In [3]:
from datetime import datetime
import pandas as pd
import numpy as np

import ipywidgets as widgets
from IPython.display import display, clear_output


In [4]:
DBNAME = "dssatserv"
con = connect(DBNAME)

In [5]:
selector_pars ={"schema": "kenya", "admin1": None, "session": None}
schema = selector_pars["schema"]

# session = Session(AdminBase(con, schema, "Bomet"))

admin1_list = admin_list(con, schema)

output_validation = widgets.Output()
def on_admin_change(change, session):
    selector_pars["session"] = Session(
        AdminBase(con, schema, change.new)
    )
    desc = selector_pars["session"].adminBase.baseline_description() + \
        ". If the assumptions are not accurate you can set your own baseline scenario."
    baseline_desc.value = f'<h4 style="padding: 0px 0px 0px 20px">{desc}</h4>'
    with output_validation:
        clear_output()
        val_chart = validation_chart(selector_pars["session"])
        val_chart.display()

admin1_picker = widgets.Dropdown(
    options=admin1_list,
    value=None,
    description='Admin1:',
    disabled=False,
)
admin1_picker.observe(lambda x: on_admin_change(x, selector_pars["schema"]), names='value')

baseline_desc = widgets.HTML(
    value=""
)


disp_box = widgets.VBox([
    widgets.HBox([admin1_picker, baseline_desc]),
    output_validation
])

selector_pars["session"]
display(disp_box)
# print(pars["adminBase"].baseline_description())

In [16]:
session = selector_pars["session"]
def change_plot_type(change):
    with output_yield:
        clear_output()
        if plot_selector.value == "Yield Range":
            Chart.from_dict(range_chart).display()
        else:
            Chart.from_dict(anom_chart).display()

plot_selector = widgets.RadioButtons(
    options=["Yield Range", "Anomaly Prob."],
    description='Chart:',
    disabled=False,
    value="Yield Range"
)
plot_selector.observe(change_plot_type, names="value")

def on_value_change(change, par):
    if par == "cultivar":
        session.simPars.__dict__[par] = session.adminBase.cultivar_labels_inv[change.new]
    else:
        session.simPars.__dict__[par] = change.new
        
plantingdate_picker = widgets.DatePicker(
    description='Planting date:',
    disabled=False,
    value=session.simPars.planting_date
)
plantingdate_picker.observe(lambda x: on_value_change(x, "planting_date"), names='value')

cultivar_picker = widgets.Dropdown(
    options=session.adminBase.cultivar_labels.values(),
    description='Cultivar:',
    disabled=False,
    value=session.adminBase.cultivar_labels[session.simPars.cultivar]
)
cultivar_picker.observe(lambda x: on_value_change(x, "cultivar"), names='value')

run_button = widgets.Button(
    description='Run DSSAT',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Run DSSAT',
    layout=widgets.Layout(width='95%',)
)
clear_button = widgets.Button(
    description='Clear',
    disabled=False,
    button_style='danger', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Clear',
    layout=widgets.Layout(width='95%',)
)

baseline_button = widgets.Button(
    description='Change Baseline',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Set Baseline',
    layout=widgets.Layout(width='95%',)
)

nitrogen_dap = widgets.IntsInput(
    value=session.simPars.nitrogen_dap,
    min=0,
    max=120,
    format='d',
    description='N date(dap):'
)
nitrogen_dap.observe(lambda x: on_value_change(x, "nitrogen_dap"), names='value')

nitrogen_rate = widgets.IntsInput(
    value=session.simPars.nitrogen_rate,
    min=0,
    max=999,
    format='d',
    description='N rate:'
)
nitrogen_rate.observe(lambda x: on_value_change(x, "nitrogen_rate"), names='value')

irrigation = widgets.Checkbox(
    value=False,
    description='Irrigation',
    disabled=True,
    indent=False
)

output_yield = widgets.Output()
output_stress1 = widgets.Output()
output_stress2 = widgets.Output()

select_box = widgets.VBox([
    widgets.HBox([
        widgets.HBox([
            widgets.VBox([run_button, clear_button, baseline_button]),
            plot_selector
        ]),
        widgets.HBox([
            widgets.VBox([plantingdate_picker, cultivar_picker]),
            widgets.VBox([
                widgets.HBox([widgets.HTML(value="Timing (dap):"), nitrogen_dap]),
                widgets.HBox([widgets.HTML(value="Rate (kg N/ha):"), nitrogen_rate]),
            ])
        ]),
    ]),
    
    output_yield,
    widgets.HBox([
        widgets.HBox([output_stress1,]),
        widgets.HBox([output_stress2,])
    ])
])

In [17]:
anom_chart = init_anomalies_chart()
range_chart = init_columnRange_chart(session)
stress_chart_1 = init_stress_chart("water")
stress_chart_2 = init_stress_chart("nitrogen")

def clear_figure(b):
    clear_yield_chart(anom_chart)
    clear_yield_chart(range_chart)
    clear_stress_chart(stress_chart_1)
    clear_stress_chart(stress_chart_2)
    with output_yield:
        clear_output()
        if plot_selector.value == "Yield Range":
            Chart.from_dict(range_chart).display()
        else:
            Chart.from_dict(anom_chart).display()
    with output_stress1:
        clear_output()
        Chart.from_dict(stress_chart_1).display()
    with output_stress2:
        clear_output()
        Chart.from_dict(stress_chart_2).display()

def on_clic_run(b):
    session.run_experiment(fakerun=True)
    with output_yield:
        # Add data for columnRangeplot
        new_chart_data = get_columnRange_series_data(session)
        for serie in range_chart["userOptions"]["series"]:
            if serie.get("data"):
                serie["data"] += [new_chart_data[serie["name"]]]
            else:
                serie["data"] = [new_chart_data[serie["name"]]]
                
        # Add data for anomaly chart
        new_chart_data = get_anomaly_series_data(session)
        for serie in anom_chart["userOptions"]["series"]:
            
            if serie.get("data"):
                serie["data"] += [new_chart_data[serie["name"]]]
            else:
                serie["data"] = [new_chart_data[serie["name"]]]     
        clear_output()
        if plot_selector.value == "Yield Range":
            Chart.from_dict(range_chart).display()
        else:
            Chart.from_dict(anom_chart).display()
            
    with output_stress1:
        # Add water stress bar
        new_chart_data = get_stress_series_data(session, stresstype="water")
        if not stress_chart_1["userOptions"].get("series"):
            new_chart_data["name"] = f"Exp 1"
            stress_chart_1["userOptions"]["series"] = [new_chart_data]
        else:
            n_exps = len(stress_chart_1["userOptions"]["series"])
            new_chart_data["name"] = f"Exp {n_exps + 1}"
            stress_chart_1["userOptions"]["series"] += [new_chart_data]
        clear_output()
        Chart.from_dict(stress_chart_1).display()
    with output_stress2:
        # Add nitrogen stress bar
        new_chart_data = get_stress_series_data(session, stresstype="nitrogen")
        if not stress_chart_2["userOptions"].get("series"):
            new_chart_data["name"] = f"Exp 1"
            stress_chart_2["userOptions"]["series"] = [new_chart_data]
        else:
            n_exps = len(stress_chart_2["userOptions"]["series"])
            new_chart_data["name"] = f"Exp {n_exps + 1}"
            stress_chart_2["userOptions"]["series"] += [new_chart_data]
        clear_output()
        Chart.from_dict(stress_chart_2).display()
        
def on_clic_baseline(b):
    session.new_baseline()
    
clear_button.on_click(clear_figure)
run_button.on_click(on_clic_run)
baseline_button.on_click(on_clic_baseline)
display(select_box, output_yield, output_stress1, output_stress2)

Output()

Output()

Output()

# STOP HERE!!!

After this it is all debugging cells so don't bother looking at them

------------------

In [55]:
chart_dict = init_stress_chart("water").to_dict()

In [56]:
chart_dict

{'userOptions': {'title': {'style': {'font-size': '15px'},
   'text': 'Water stress'},
  'tooltip': {'headerFormat': '<span style="font-size: 12px; font-weight: bold">{point.key}</span><br/>',
   'pointFormat': '<span style="color:{point.color};font-size: 12px">● </span><span style="font-size: 12px">{series.name}: {point.y:.0f} %</span><br/>'},
  'xAxis': {'categories': ['Emerg.-End<br>Juv.',
    'End Juv-<br>Flor Init',
    'Flor Init-<br>End Lf Gro',
    'End lf Gro-<br>Beg Grain<br>Fil',
    'Grain<br>Fill'],
   'labels': {'allowOverlap': True,
    'autoRotationLimit': 0,
    'style': {'font-size': '15px'}},
   'title': {'style': {'font-size': '15px'}, 'text': 'Crop Dev. Stage'}},
  'yAxis': {'labels': {'style': {'font-size': '15px'}},
   'max': 100,
   'title': {'style': {'font-size': '15px'}, 'text': 'Stress (%)'}}}}

In [57]:
from dssatservice.ui.plot import process_overview, STRESS_COLUMNS, ColumnSeries, DEV_STAGES
def get_stress_series_data(session, stresstype="water"):
    """
    data is pandas.Series with the dev stage as index. stresstype is one among
    water or nitrogen
    """
    overview = process_overview(session.latest_overview.copy())
    var_column = STRESS_COLUMNS[stresstype]
    data = overview.groupby("devPhase")[var_column].mean()
    data = 100*data
    box = ColumnSeries()
    box.data = [data.to_dict().get(dev_st) for dev_st in DEV_STAGES]
    return box.to_dict()

In [58]:
get_stress_series_data(session)

{'data': [{'y': 0.0},
  {'y': 0.0},
  {'y': 4.94},
  {'y': 0.9059999999999999},
  {'y': 12.659999999999998}],
 'type': 'column'}

In [64]:
new_chart_data = get_stress_series_data(session)
if not chart_dict["userOptions"].get("series"):
    new_chart_data["name"] = f"Exp 1"
    chart_dict["userOptions"]["series"] = [new_chart_data]
else:
    n_exps = len(chart_dict["userOptions"]["series"])
    new_chart_data["name"] = f"Exp {n_exps + 1}"
    chart_dict["userOptions"]["series"] += [new_chart_data]

In [65]:
chart = Chart.from_dict(chart_dict)
chart.display()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [9]:
chart = range_chart.copy()
chart.options.y_axis = {
    "title": {
        'text': 'Experiment', 
        "style": {
            "font-size": "15px",
        }
    },
    "labels": {
        "style": {
            "font-size": "15px",
        }
    },
    "plot_lines": [
        {
            "value": session.adminBase.validation_run.obs.mean(),
            "color": "#00000032",
            "width": 10,
            "dash_style": "Solid",
            "z_index": 99,
            "label": {
                "text": f"<b>County<br/>average</b>",
                "align": "left",
                "style": {"color": "black", "font-size": 13}
            }
        },
        {
            "value": session.adminBase.validation_run.obs.min(),
            "color": "#32323232",
            "width": 3,
            "dash_style": "Dash",
            "z_index": 0,
            "label": {
                "text": f"County<br/>min",
                "align": "left",
                "style": {"color": "black", "font-size": 13}
            }
        },
        {
            "value": session.adminBase.validation_run.obs.max(),
            "color": "#32323232",
            "width": 3,
            "dash_style": "Dash",
            "z_index": 0,
            "label": {
                "text": f"<b>County<br/>max</b>",
                "align": "left",
                "style": {"color": "black", "font-size": 13}
            }
        }
    ]
}
chart.display()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [60]:
session.adminBase.schema

'kenya'

In [24]:
session.experiment_results

,planting,nitro_dap,nitro_rate,cultivar,yield_range,harvest_range
0,2021-04-01 00:00:00,"(5, 30, 60)","(21.414078312364648, 21.414078312364648, 21.41...",IF0009,"(4.0423, 6.5222999999999995)","(154, 278)"
1,2020-04-01,"(5, 30, 60)","(21.414078312364648, 21.414078312364648, 21.41...",IF0009,"(2.7819, 6.63265)","(154, 224)"


In [37]:
# with open("/home/dquintero/OVERVIEW.OUT", "w") as f:
#     f.writelines(session.latest_overview)
session.latest_overview = 

['*SIMULATION OVERVIEW FILE\n',
 '\n',
 '*DSSAT Cropping System Model Ver. 4.8.2.000 -release  JUN 13, 2024 20:54:37\n',
 '\n',
 '*RUN   1        : SAMPLE 1                  MZCER048 EXPEFILE    1\n',
 ' MODEL          : MZCER048 - Maize\n',
 ' EXPERIMENT     : EXPEFILE MZ SPATIAL ANALYSES TEST CASE; FLORENCE, SOUTH CAROLI\n',
 ' DATA PATH      : /tmp/dssatrun7XN8CCF9/\n',
 ' TREATMENT  1   : SAMPLE 1                  MZCER048\n',
 '\n',
 '\n',
 ' CROP           : Maize            CULTIVAR : SOTUBAKA         ECOTYPE :IB0001\n',
 ' STARTING DATE  : APR  1 2021\n',
 ' PLANTING DATE  : AUTOMATIC PLANTING PLANTS/m2 :  4.0     ROW SPACING :  90.cm\n',
 ' WEATHER        : SEAA   2021\n',
 ' SOIL           : IB00000001     TEXTURE : yLoam -    ISRIC soilgrids + HC27\n',
 ' SOIL INIT COND : DEPTH:200cm EXTR. H2O:252.5mm  NO3:  0.0kg/ha  NH4:  0.0kg/ha\n',
 ' WATER BALANCE  : RAINFED\n',
 ' IRRIGATION     : NOT IRRIGATED\n',
 ' NITROGEN BAL.  : SOIL-N & N-UPTAKE SIMULATION; NO N-FIXATION\n',
 '

array([357.54580636, 674.66102708, 636.73081364, 610.68663046,
       606.0118816 , 621.74775001, 557.92684485, 586.77484631,
       536.33324831, 486.58136258, 567.39672334, 434.32707078,
       483.77977457, 626.42485852, 552.8385661 , 681.58905947,
       569.95311194, 569.17782088, 289.8461305 , 473.90123825,
       573.18066032, 477.43617891, 602.70517101, 556.01074319,
       521.29445992, 521.53948127, 688.88324067, 650.05767453,
       430.87481993, 437.90370403, 594.94461215, 624.41250239,
       597.53808189, 591.21930244, 377.40218493, 705.72868703,
       525.4562077 , 423.88171376, 371.06327862, 598.17239014,
       307.74768458, 587.19256829, 434.84291051, 374.99076149,
       505.49971923, 398.26710393, 471.0812213 , 550.43472694,
       278.15221879, 794.48135091])

In [30]:
pd.DataFrame({
    "HARWT": (np.random.normal(0, 1, 50)*100) + 500
})

,HARWT
0,475.999563
1,656.765031
2,524.401523
3,375.300447
4,453.136185
5,583.366560
6,406.477194
7,514.277387
8,419.102705
9,576.338348


In [10]:
session.latest_run

,RUN,CR,TRT,FLO,MAT,TOPWT,HARWT,RAIN,TIRR,CET,PESW,TNUP,TNLF,TSON,TSOC,year,sim
0,1,MZ,1,92,160,3734,2024,1727,0,593,291,39,26,15174,155,1,2.024
1,2,MZ,2,66,110,3702,1894,658,0,451,197,43,27,16335,165,1,1.894
2,3,MZ,3,61,99,3207,1654,635,0,430,202,36,23,11474,118,1,1.654
3,4,MZ,4,65,107,4941,2311,460,0,454,171,55,28,20686,209,1,2.311
4,5,MZ,5,72,122,4542,2317,680,0,459,178,53,32,22031,222,1,2.317
5,6,MZ,6,62,100,4390,2071,636,0,434,175,46,27,16664,169,1,2.071
6,7,MZ,7,62,100,4404,2074,636,0,434,175,46,27,16802,170,1,2.074
7,8,MZ,8,61,99,4594,2079,635,0,435,175,52,29,20683,209,1,2.079
8,9,MZ,9,66,110,4236,2153,658,0,448,177,42,26,15179,155,1,2.153
9,10,MZ,10,61,99,4096,1962,635,0,432,176,38,25,13358,135,1,1.962


In [15]:
session.adminBase.baseline_run

,year,sim,obs
300,2017,1.105,NaN
301,2017,0.202,NaN
302,2017,1.363,NaN
303,2017,0.813,NaN
304,2017,0.368,NaN
...,...,...,...
545,2021,0.382,NaN
546,2021,0.252,NaN
547,2021,0.666,NaN
548,2021,0.472,NaN


In [9]:
session.adminBase.baseline_run.groupby("year").obs.mean()

year
2017         NaN
2018         NaN
2019    1.897805
2020         NaN
2021         NaN
Name: obs, dtype: float64

In [12]:
session.adminBase.validation_run.groupby("year").obs.mean().

year
2011    0.410907
2012    0.963000
2013    0.981000
2014    0.819000
Name: obs, dtype: float64

In [8]:
session.adminBase.baseline_run

,year,sim,obs
300,2017,1.105,NaN
301,2017,0.202,NaN
302,2017,1.363,NaN
303,2017,0.813,NaN
304,2017,0.368,NaN
...,...,...,...
545,2021,0.382,NaN
546,2021,0.252,NaN
547,2021,0.666,NaN
548,2021,0.472,NaN


In [22]:
session.adminBase.admin1

'Garissa'

In [19]:
!pip freeze | grep widgetsnbextension

widgetsnbextension==4.0.9
